In [1]:
import itertools
import time
import re
from IPython.display import clear_output
import random

In [2]:
WIDTH = 50
HEIGHT = 8

In [3]:
def new_grid(w=WIDTH, h=HEIGHT):
    return ['.' * w for r in range(1, h+1)]

In [4]:
def print_grid(grid, md=False, suppress_dots=False):
    if md:
        print('```')
    for row in grid:
        if suppress_dots:
            print(re.sub(r'\.', ' ', row))
        else:
            print(row)        
    if md:
        print('```')

In [5]:
def top(grid, l, r):
    new_segment = ''
    for i in range(l-1, r):
        if grid[0][i] == '.':
            new_segment += '*'
        else:
            new_segment += '.'
    grid[0] = grid[0][:l-1] + new_segment + grid[0][r:]
    return grid

In [6]:
def left(grid, t, b):
    for i in range(t-1, b):
        if grid[i][0] == '.':
            grid[i] = '*' + grid[i][1:]
        else:
            grid[i] = '.' + grid[i][1:]
    return grid

In [7]:
def rotate_column(grid, c, raw_n):
    n = raw_n % len(grid)
    col = [row[c-1] for row in grid]
    new_col = col[-n:] + col[:-n]
    for i in range(len(grid)):
        grid[i] = grid[i][:c-1] + new_col[i] + grid[i][c:]
    return grid

In [8]:
def rotate_row(grid, r, raw_n):
    n = raw_n % len(grid[0])
    grid[r-1] = grid[r-1][-n:] + grid[r-1][:-n]
    return grid

In [9]:
command_dispatch = {'left': left, 'top': top,
                   'rotate row': rotate_row,
                   'rotate column': rotate_column}

In [10]:
def parse(command):
    cmd, a, b = command.rsplit(maxsplit=2)
    return cmd, int(a), int(b)  

In [11]:
def interpret(commands, grid=None, w=WIDTH, h=HEIGHT, 
              uninterpret=False,
              show_each_step=False, md=False, overprint=False,
              suppress_dots=False):
    if grid is None:
        grid = new_grid(w, h)
    if uninterpret:
        ordered_commands = reversed(commands)
    else:
        ordered_commands = commands
    for c in ordered_commands:
        cmd, a, b = parse(c)
        if uninterpret and cmd in uncommand_dispatch:
            uncommand_dispatch[cmd](grid, a, b)
        elif not uninterpret and cmd in command_dispatch:
            command_dispatch[cmd](grid, a, b)
        else:
            raise ValueError('Unknown command')
        if show_each_step:
            if overprint:
                time.sleep(0.25)
            if md: 
                print('`{}`'.format(c))
            else:
                print(c)
            print_grid(grid, md=md, suppress_dots=suppress_dots)
            print()
            if overprint:
                clear_output(wait=True)
    if show_each_step: 
        print('Final')
        print_grid(grid, md=md, suppress_dots=suppress_dots)
    return grid
    
    
# for i in range(10):
#     time.sleep(0.25)
#     print(i)
#     clear_output(wait=True)

For instance, with a smaller grid that is 10 pixels wide and 4 tall, this is what a sample sequence of instructions would do.

* `toggle 1 6` turns on the first six pixels on the top row.
```
******....
..........
..........
..........
```

* `rotate column 2 3` moves the lit pixel on the second column to the bottom row.
```
*.****....
..........
..........
.*........
```

* `toggle 3 10` turns off the pixels in columns 4, 5, and 6, and turns on the pixels in columns 7 to 10.

```
*.....****
..........
..........
.*........
```

* `rotate column 8 1` moves the one lit pixel in column 8 down one row.
```
*.....*.**
.......*..
..........
.*........
```

* `rotate row 2 6` moves that pixel off the right edge of the display, to it wraps around to appear in column 4.
```
*.....*.**
...*......
..........
.*........
```

* `left 1 3` toggles the pixels in rows 1, 2, and 3 of the first column. The top left pixel (previously on) turns off, while the pixels in rows 2 and 3 come on.

```
......*.**
*..*......
*.........
.*........
```

In [12]:
cmds = '''
top 1 6
rotate column 2 3
top 3 10
rotate column 8 1
rotate row 2 6
left 1 3
'''.split('\n')[1:-1]
interpret(cmds, w=10, h=4, show_each_step=True, md=True)

`top 1 6`
```
******....
..........
..........
..........
```

`rotate column 2 3`
```
*.****....
..........
..........
.*........
```

`top 3 10`
```
*.....****
..........
..........
.*........
```

`rotate column 8 1`
```
*.....*.**
.......*..
..........
.*........
```

`rotate row 2 6`
```
*.....*.**
...*......
..........
.*........
```

`left 1 3`
```
......*.**
*..*......
*.........
.*........
```

Final
```
......*.**
*..*......
*.........
.*........
```


['......*.**', '*..*......', '*.........', '.*........']

In [13]:
def unrotate_column(grid, c, raw_n):
    return rotate_column(grid, c, (-1 * raw_n) % len(grid))

def unrotate_row(grid, r, raw_n):
    return rotate_row(grid, r, (-1 * raw_n) % len(grid[0]))


In [14]:
uncommand_dispatch = {'left': left, 'top': top,
                      'rotate row': unrotate_row,
                      'rotate column': unrotate_column}

In [15]:
trantor_grid = '''
..................................................
..*****..****....***...*...*..*****...***...****..
....*....*...*..*...*..**..*....*....*...*..*...*.
....*....*..*...*...*..*.*.*....*....*...*..*..*..
....*....****...*****..*.*.*....*....*...*..****..
....*....*...*..*...*..*..**....*....*...*..*...*.
....*....*...*..*...*..*...*....*.....***...*...*.
..................................................
'''.split('\n')[1:-1]
trantor_grid

['..................................................',
 '..*****..****....***...*...*..*****...***...****..',
 '....*....*...*..*...*..**..*....*....*...*..*...*.',
 '....*....*..*...*...*..*.*.*....*....*...*..*..*..',
 '....*....****...*****..*.*.*....*....*...*..****..',
 '....*....*...*..*...*..*..**....*....*...*..*...*.',
 '....*....*...*..*...*..*...*....*.....***...*...*.',
 '..................................................']

In [16]:
def tg():
    return '''
..................................................
..*****..****....***...*...*..*****...***...****..
....*....*...*..*...*..**..*....*....*...*..*...*.
....*....*..*...*...*..*.*.*....*....*...*..*..*..
....*....****...*****..*.*.*....*....*...*..****..
....*....*...*..*...*..*..**....*....*...*..*...*.
....*....*...*..*...*..*...*....*.....***...*...*.
..................................................
'''.split('\n')[1:-1]

In [17]:
print_grid(new_grid(100, 8))

....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................


In [18]:
def jantar_mantar():
    return '''
..*****.............................................................................................
.....*..............................................................................................
.....*..............................................................................................
.....*..............................................................................................
.....*..............................................................................................
.....*..............................................................................................
.*...*..............................................................................................
..***...............................................................................................
'''.split('\n')[1:-1]

In [19]:
def jantar_mantar():
    return '''
...****..............*...................*.....*..............*.................
......*..............*...................***..**..............*.................
......*.*****.*****.****.*****..****.....*.*.***.*****.*****.****.*****..****...
......*.....*.*...*..*.......*..*........*..**.*.....*.*...*..*.......*..*......
......*.*****.*...*..*...*****..*........*..*..*.*****.*...*..*...*****..*......
......*.*...*.*...*..*...*...*..*........*.....*.*...*.*...*..*...*...*..*......
...*..*.*..**.*...*..**..*..**..*........*.....*.*..**.*...*..**..*..**..*......
....**...**.*.*...*...**..**.*..*........*.....*..**.*.*...*...**..**.*..*......
'''.split('\n')[1:-1]

In [20]:
trantor_grid = '''
..................................................
..*****..****....***...*...*..*****...***...****..
....*....*...*..*...*..**..*....*....*...*..*...*.
....*....*..*...*...*..*.*.*....*....*...*..*..*..
....*....****...*****..*.*.*....*....*...*..****..
....*....*...*..*...*..*..**....*....*...*..*...*.
....*....*...*..*...*..*...*....*.....***...*...*.
..................................................
'''.split('\n')[1:-1]

cmds = '''
rotate column 3 1
left 3 7
rotate column 1 1
left 3 7
'''.split('\n')[1:-1]
interpret(cmds, trantor_grid, show_each_step=True, uninterpret=True)

left 3 7
..................................................
..*****..****....***...*...*..*****...***...****..
*...*....*...*..*...*..**..*....*....*...*..*...*.
*...*....*..*...*...*..*.*.*....*....*...*..*..*..
*...*....****...*****..*.*.*....*....*...*..****..
*...*....*...*..*...*..*..**....*....*...*..*...*.
*...*....*...*..*...*..*...*....*.....***...*...*.
..................................................

rotate column 1 1
..................................................
*.*****..****....***...*...*..*****...***...****..
*...*....*...*..*...*..**..*....*....*...*..*...*.
*...*....*..*...*...*..*.*.*....*....*...*..*..*..
*...*....****...*****..*.*.*....*....*...*..****..
*...*....*...*..*...*..*..**....*....*...*..*...*.
....*....*...*..*...*..*...*....*.....***...*...*.
..................................................

left 3 7
..................................................
*.*****..****....***...*...*..*****...***...****..
....*....*...*..*...*..**..*....*....*...*..

['..*...............................................',
 '*..****..****....***...*...*..*****...***...****..',
 '....*....*...*..*...*..**..*....*....*...*..*...*.',
 '....*....*..*...*...*..*.*.*....*....*...*..*..*..',
 '....*....****...*****..*.*.*....*....*...*..****..',
 '....*....*...*..*...*..*..**....*....*...*..*...*.',
 '*...*....*...*..*...*..*...*....*.....***...*...*.',
 '..................................................']

In [21]:
def transpose(grid):
    return list(zip(*grid))

In [22]:
transpose(trantor_grid)

[('.', '*', '.', '.', '.', '.', '*', '.'),
 ('.', '.', '.', '.', '.', '.', '.', '.'),
 ('*', '.', '.', '.', '.', '.', '.', '.'),
 ('.', '*', '.', '.', '.', '.', '.', '.'),
 ('.', '*', '*', '*', '*', '*', '*', '.'),
 ('.', '*', '.', '.', '.', '.', '.', '.'),
 ('.', '*', '.', '.', '.', '.', '.', '.'),
 ('.', '.', '.', '.', '.', '.', '.', '.'),
 ('.', '.', '.', '.', '.', '.', '.', '.'),
 ('.', '*', '*', '*', '*', '*', '*', '.'),
 ('.', '*', '.', '.', '*', '.', '.', '.'),
 ('.', '*', '.', '.', '*', '.', '.', '.'),
 ('.', '*', '.', '*', '*', '.', '.', '.'),
 ('.', '.', '*', '.', '.', '*', '*', '.'),
 ('.', '.', '.', '.', '.', '.', '.', '.'),
 ('.', '.', '.', '.', '.', '.', '.', '.'),
 ('.', '.', '*', '*', '*', '*', '*', '.'),
 ('.', '*', '.', '.', '*', '.', '.', '.'),
 ('.', '*', '.', '.', '*', '.', '.', '.'),
 ('.', '*', '.', '.', '*', '.', '.', '.'),
 ('.', '.', '*', '*', '*', '*', '*', '.'),
 ('.', '.', '.', '.', '.', '.', '.', '.'),
 ('.', '.', '.', '.', '.', '.', '.', '.'),
 ('.', '*',

In [23]:
def count_empty_rows(grid, col):
    return len(list(itertools.takewhile(lambda c: c == '.', transpose(grid)[col-1])))

In [24]:
[count_empty_rows(trantor_grid, i) for i in range(1, WIDTH+1)]

[1,
 8,
 0,
 1,
 1,
 1,
 1,
 8,
 8,
 1,
 1,
 1,
 1,
 2,
 8,
 8,
 2,
 1,
 1,
 1,
 2,
 8,
 8,
 1,
 2,
 3,
 5,
 1,
 8,
 8,
 1,
 1,
 1,
 1,
 1,
 8,
 8,
 2,
 1,
 1,
 1,
 2,
 8,
 8,
 1,
 1,
 1,
 1,
 2,
 8]

In [25]:
def lift_cols(grid, randomise=False):
    commands = []
    h = len(grid)
    for i in range(1, len(grid[0])+1):
        n = count_empty_rows(grid, i)
        if n != h:
            commands.append('rotate column {} {}'.format(i, n))
    if randomise:
        random.shuffle(commands)
    return commands

In [26]:
lift_cols(tg())

['rotate column 3 1',
 'rotate column 4 1',
 'rotate column 5 1',
 'rotate column 6 1',
 'rotate column 7 1',
 'rotate column 10 1',
 'rotate column 11 1',
 'rotate column 12 1',
 'rotate column 13 1',
 'rotate column 14 2',
 'rotate column 17 2',
 'rotate column 18 1',
 'rotate column 19 1',
 'rotate column 20 1',
 'rotate column 21 2',
 'rotate column 24 1',
 'rotate column 25 2',
 'rotate column 26 3',
 'rotate column 27 5',
 'rotate column 28 1',
 'rotate column 31 1',
 'rotate column 32 1',
 'rotate column 33 1',
 'rotate column 34 1',
 'rotate column 35 1',
 'rotate column 38 2',
 'rotate column 39 1',
 'rotate column 40 1',
 'rotate column 41 1',
 'rotate column 42 2',
 'rotate column 45 1',
 'rotate column 46 1',
 'rotate column 47 1',
 'rotate column 48 1',
 'rotate column 49 2']

In [27]:
lift_cols(tg(), randomise=True)

['rotate column 24 1',
 'rotate column 41 1',
 'rotate column 10 1',
 'rotate column 18 1',
 'rotate column 39 1',
 'rotate column 3 1',
 'rotate column 6 1',
 'rotate column 14 2',
 'rotate column 42 2',
 'rotate column 32 1',
 'rotate column 20 1',
 'rotate column 38 2',
 'rotate column 34 1',
 'rotate column 31 1',
 'rotate column 35 1',
 'rotate column 28 1',
 'rotate column 45 1',
 'rotate column 13 1',
 'rotate column 40 1',
 'rotate column 5 1',
 'rotate column 4 1',
 'rotate column 12 1',
 'rotate column 47 1',
 'rotate column 25 2',
 'rotate column 7 1',
 'rotate column 49 2',
 'rotate column 46 1',
 'rotate column 26 3',
 'rotate column 27 5',
 'rotate column 19 1',
 'rotate column 11 1',
 'rotate column 17 2',
 'rotate column 48 1',
 'rotate column 33 1',
 'rotate column 21 2']

In [28]:
def count_empty_cols(grid, row):
    return len(list(itertools.takewhile(lambda r: r == '.', grid[row-1])))

In [29]:
[count_empty_cols(trantor_grid, i) for i in range(1, HEIGHT+1)]

[2, 0, 4, 4, 4, 4, 0, 50]

In [30]:
def slide_rows(grid, randomise=False):
    commands = []
    w = len(grid[0])
    for i in range(1, len(grid)+1):
        n = count_empty_cols(grid, i)
        if n != w:
            commands.append('rotate row {} {}'.format(i, n))
    if randomise:
        random.shuffle(commands)
    return commands

In [31]:
def untop(grid, randomise=False):
    groups = [(k, len(list(g))) for k, g in itertools.groupby(grid[0])]
    commands = []
    col = 1
    for c, l in groups:
        if c == '*':
            commands.append('top {} {}'.format(col, col + l - 1))
        col += l
    if randomise:
        random.shuffle(commands)
    return commands

In [32]:
def unleft(grid, randomise=False):
    groups = [(k, len(list(g))) for k, g in itertools.groupby(transpose(grid)[0])]
    commands = []
    row = 1
    for c, l in groups:
        if c == '*':
            commands.append('left {} {}'.format(row, row + l - 1))
        row += l
    if randomise:
        random.shuffle(commands)
    return commands

In [33]:
trantor_grid = tg() 

cmds = []

c = slide_rows(trantor_grid)
cmds = c + cmds

interpret(c, trantor_grid, uninterpret=True)

c = lift_cols(trantor_grid)
cmds = c + cmds

interpret(c, trantor_grid, uninterpret=True)

['***********.******.*****.*..***********.******....',
 '*....*...*..*...*..*...*....*....*...*..*...*.....',
 '*....*..*...*...*..*.*.*....*....*...*..*..*......',
 '*....*.**...*..**..*.*.*....*....*...*..*.**......',
 '*....*...*..*...*..*...*....*........*..*...*.....',
 '*........*......*...........*.......*.......*.....',
 '..................................................',
 '..................................................']

In [34]:
g = interpret(cmds, tg(), uninterpret=True)
untop(g)

['top 1 11', 'top 13 18', 'top 20 24', 'top 26 26', 'top 29 39', 'top 41 46']

In [35]:
g = tg() 

cmds = []

c = slide_rows(g, randomise=True)
interpret(c, g, uninterpret=True)
cmds = c + cmds

c = lift_cols(g, randomise=True)
interpret(c, g, uninterpret=True)
cmds = c + cmds

c = untop(g, randomise=True)
interpret(c, g, uninterpret=True)
cmds = c + cmds

c = unleft(g, randomise=True)
interpret(c, g, uninterpret=True)
cmds = c + cmds

print_grid(g)

..................................................
.....*...*..*...*..*...*....*....*...*..*...*.....
.....*..*...*...*..*.*.*....*....*...*..*..*......
.....*.**...*..**..*.*.*....*....*...*..*.**......
.....*...*..*...*..*...*....*........*..*...*.....
.........*......*...........*.......*.......*.....
..................................................
..................................................


In [36]:
g = tg() 
cmds = []

while '*' in ''.join(g):
    if random.choice([True, False]):
        c = slide_rows(g, randomise=True)
        interpret(c, g, uninterpret=True)
        cmds = c + cmds
    
        c = unleft(g, randomise=True)
        interpret(c, g, uninterpret=True)
        cmds = c + cmds
        
    else:
        c = lift_cols(g, randomise=True)
        interpret(c, g, uninterpret=True)
        cmds = c + cmds
    
        c = untop(g, randomise=True)
        interpret(c, g, uninterpret=True)
        cmds = c + cmds
    
print_grid(g)
len(cmds)

..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................


139

In [37]:
g = tg()
interpret(cmds, g, uninterpret=True)

['..................................................',
 '..................................................',
 '..................................................',
 '..................................................',
 '..................................................',
 '..................................................',
 '..................................................',
 '..................................................']

In [38]:
interpret(cmds)

['..................................................',
 '..*****..****....***...*...*..*****...***...****..',
 '....*....*...*..*...*..**..*....*....*...*..*...*.',
 '....*....*..*...*...*..*.*.*....*....*...*..*..*..',
 '....*....****...*****..*.*.*....*....*...*..****..',
 '....*....*...*..*...*..*..**....*....*...*..*...*.',
 '....*....*...*..*...*..*...*....*.....***...*...*.',
 '..................................................']

In [39]:
cmds

['top 5 5',
 'top 8 8',
 'top 29 29',
 'top 12 12',
 'rotate column 29 1',
 'rotate column 8 1',
 'rotate column 12 1',
 'rotate column 5 1',
 'top 12 12',
 'top 22 22',
 'top 17 17',
 'top 29 29',
 'top 5 5',
 'top 26 26',
 'top 8 8',
 'rotate column 5 1',
 'rotate column 26 1',
 'rotate column 12 1',
 'rotate column 29 1',
 'rotate column 8 1',
 'rotate column 22 1',
 'rotate column 17 1',
 'top 21 22',
 'top 29 30',
 'top 5 5',
 'top 2 2',
 'top 33 33',
 'top 14 14',
 'top 16 17',
 'top 37 37',
 'top 12 12',
 'top 26 26',
 'top 8 9',
 'rotate column 30 1',
 'rotate column 5 1',
 'rotate column 22 2',
 'rotate column 33 1',
 'rotate column 12 1',
 'rotate column 21 1',
 'rotate column 16 1',
 'rotate column 2 1',
 'rotate column 14 1',
 'rotate column 9 1',
 'rotate column 37 1',
 'rotate column 26 1',
 'rotate column 8 2',
 'rotate column 17 2',
 'rotate column 29 2',
 'left 2 5',
 'rotate row 2 3',
 'rotate row 4 7',
 'rotate row 5 7',
 'rotate row 3 7',
 'top 19 19',
 'top 29 33',

In [40]:
g = new_grid()
interpret(cmds, g, show_each_step=True, overprint=True)

Final
..................................................
..*****..****....***...*...*..*****...***...****..
....*....*...*..*...*..**..*....*....*...*..*...*.
....*....*..*...*...*..*.*.*....*....*...*..*..*..
....*....****...*****..*.*.*....*....*...*..****..
....*....*...*..*...*..*..**....*....*...*..*...*.
....*....*...*..*...*..*...*....*.....***...*...*.
..................................................


['..................................................',
 '..*****..****....***...*...*..*****...***...****..',
 '....*....*...*..*...*..**..*....*....*...*..*...*.',
 '....*....*..*...*...*..*.*.*....*....*...*..*..*..',
 '....*....****...*****..*.*.*....*....*...*..****..',
 '....*....*...*..*...*..*..**....*....*...*..*...*.',
 '....*....*...*..*...*..*...*....*.....***...*...*.',
 '..................................................']

In [41]:
g = tg()
interpret(cmds, g, show_each_step=True, overprint=True, uninterpret=True)

Final
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................


['..................................................',
 '..................................................',
 '..................................................',
 '..................................................',
 '..................................................',
 '..................................................',
 '..................................................',
 '..................................................']

In [42]:
sum(1 for c in ''.join(tg()) if c == '*')

98

In [43]:
open('05-pixels.txt', 'w').write('\n'.join(cmds))

2163

In [44]:
g = jantar_mantar() 
cmds = []

while '*' in ''.join(g):
    if random.choice([True, False]):
        c = slide_rows(g, randomise=True)
        interpret(c, g, uninterpret=True)
        cmds = c + cmds
    
        c = unleft(g, randomise=True)
        interpret(c, g, uninterpret=True)
        cmds = c + cmds
        
    else:
        c = lift_cols(g, randomise=True)
        interpret(c, g, uninterpret=True)
        cmds = c + cmds
    
        c = untop(g, randomise=True)
        interpret(c, g, uninterpret=True)
        cmds = c + cmds
    
print_grid(g)
len(cmds)

................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................


237

In [45]:
g = new_grid(w=80)
interpret(cmds, g, show_each_step=True, overprint=True)

Final
...****..............*...................*.....*..............*.................
......*..............*...................***..**..............*.................
......*.*****.*****.****.*****..****.....*.*.***.*****.*****.****.*****..****...
......*.....*.*...*..*.......*..*........*..**.*.....*.*...*..*.......*..*......
......*.*****.*...*..*...*****..*........*..*..*.*****.*...*..*...*****..*......
......*.*...*.*...*..*...*...*..*........*.....*.*...*.*...*..*...*...*..*......
...*..*.*..**.*...*..**..*..**..*........*.....*.*..**.*...*..**..*..**..*......
....**...**.*.*...*...**..**.*..*........*.....*..**.*.*...*...**..**.*..*......


['...****..............*...................*.....*..............*.................',
 '......*..............*...................***..**..............*.................',
 '......*.*****.*****.****.*****..****.....*.*.***.*****.*****.****.*****..****...',
 '......*.....*.*...*..*.......*..*........*..**.*.....*.*...*..*.......*..*......',
 '......*.*****.*...*..*...*****..*........*..*..*.*****.*...*..*...*****..*......',
 '......*.*...*.*...*..*...*...*..*........*.....*.*...*.*...*..*...*...*..*......',
 '...*..*.*..**.*...*..**..*..**..*........*.....*.*..**.*...*..**..*..**..*......',
 '....**...**.*.*...*...**..**.*..*........*.....*..**.*.*...*...**..**.*..*......']

In [46]:
len(jantar_mantar()[0])

80

In [47]:
open('05-pixels.txt', 'w').write('\n'.join(cmds))

3900

In [48]:
print_grid(jantar_mantar(), suppress_dots=True)

   ****              *                   *     *              *                 
      *              *                   ***  **              *                 
      * ***** ***** **** *****  ****     * * *** ***** ***** **** *****  ****   
      *     * *   *  *       *  *        *  ** *     * *   *  *       *  *      
      * ***** *   *  *   *****  *        *  *  * ***** *   *  *   *****  *      
      * *   * *   *  *   *   *  *        *     * *   * *   *  *   *   *  *      
   *  * *  ** *   *  **  *  **  *        *     * *  ** *   *  **  *  **  *      
    **   ** * *   *   **  ** *  *        *     *  ** * *   *   **  ** *  *      


In [49]:
g = new_grid(w=80)
interpret(cmds, g, show_each_step=True, overprint=True, suppress_dots=True);

Final
   ****              *                   *     *              *                 
      *              *                   ***  **              *                 
      * ***** ***** **** *****  ****     * * *** ***** ***** **** *****  ****   
      *     * *   *  *       *  *        *  ** *     * *   *  *       *  *      
      * ***** *   *  *   *****  *        *  *  * ***** *   *  *   *****  *      
      * *   * *   *  *   *   *  *        *     * *   * *   *  *   *   *  *      
   *  * *  ** *   *  **  *  **  *        *     * *  ** *   *  **  *  **  *      
    **   ** * *   *   **  ** *  *        *     *  ** * *   *   **  ** *  *      
